In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from sklearn import cross_validation
from sklearn import preprocessing

np.random.seed(15)

#read in datasets
teams = pd.read_csv('Teams.csv')
seasons = pd.read_csv('Seasons.csv')
tourneyCompactResults = pd.read_csv('TourneyCompactResults.csv')
tourneyDetailedResults = pd.read_csv('TourneyDetailedResults.csv')
tourneySeeds = pd.read_csv('TourneySeeds.csv')
tourneySlots = pd.read_csv('TourneySlots.csv')
regularSeasonCompactResults = pd.read_csv('RegularSeasonCompactResults.csv')
regularSeasonDetailedResults = pd.read_csv('RegularSeasonDetailedResults.csv')
submission_df = pd.read_csv( 'sample_submission.csv')

#turn all datasets into DataFrames
teams_df = pd.DataFrame(teams)
seasons_df = pd.DataFrame(seasons)
tourneyCompact_df = pd.DataFrame(tourneyCompactResults)
tourneyDetailed_df = pd.DataFrame(tourneyDetailedResults)
tourneySeeds_df = pd.DataFrame(tourneySeeds)
tourneySlots_df = pd.DataFrame(tourneySlots)
regSeasonCompact_df = pd.DataFrame(regularSeasonCompactResults)
regSeasonDetailed_df = pd.DataFrame(regularSeasonDetailedResults)

In [49]:
teams_df.head()

Team_Id    Team_Name
0     1101  Abilene Chr
1     1102    Air Force
2     1103        Akron
3     1104      Alabama
4     1105  Alabama A&M

In [50]:
seasons_df.head()

Season     Dayzero Regionw    Regionx    Regiony    Regionz
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988   11/2/1987    East    Midwest  Southeast       West
4    1989  10/31/1988    East       West    Midwest  Southeast

In [51]:
tourneyCompact_df.head()

Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985     136   1116      63   1234      54    N      0
1    1985     136   1120      59   1345      58    N      0
2    1985     136   1207      68   1250      43    N      0
3    1985     136   1229      58   1425      55    N      0
4    1985     136   1242      49   1325      38    N      0

In [52]:
tourneyDetailed_df.head()

Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003     134   1421      92   1411      84    N      1    32    69 ...    
1    2003     136   1112      80   1436      51    N      0    31    66 ...    
2    2003     136   1113      84   1272      71    N      0    31    59 ...    
3    2003     136   1141      79   1166      73    N      0    29    53 ...    
4    2003     136   1143      76   1301      74    N      1    27    64 ...    

   Lfga3  Lftm  Lfta  Lor  Ldr  Last  Lto  Lstl  Lblk  Lpf  
0     31    14    31   17   28    16   15     5     0   22  
1     16     7     7    8   26    12   17    10     3   15  
2     28    14    21   20   22    11   12     2     5   18  
3     17    12    17   14   17    20   21     6     6   21  
4     21    15    20   10   26    16   14     5     8   19  

[5 rows x 34 columns]

In [53]:
tourneySeeds_df.head()

Season Seed  Team
0    1985  W01  1207
1    1985  W02  1210
2    1985  W03  1228
3    1985  W04  1260
4    1985  W05  1374

In [54]:
tourneySlots_df.head()

Season  Slot Strongseed Weakseed
0    1985  R1W1        W01      W16
1    1985  R1W2        W02      W15
2    1985  R1W3        W03      W14
3    1985  R1W4        W04      W13
4    1985  R1W5        W05      W12

In [55]:
regSeasonCompact_df.head()

Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985      20   1228      81   1328      64    N      0
1    1985      25   1106      77   1354      70    H      0
2    1985      25   1112      63   1223      56    H      0
3    1985      25   1165      70   1432      54    H      0
4    1985      25   1192      86   1447      74    H      0

In [56]:
regSeasonDetailed_df.head()

Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003      10   1104      68   1328      62    N      0    27    58 ...    
1    2003      10   1272      70   1393      63    N      0    26    62 ...    
2    2003      11   1266      73   1437      61    N      0    24    58 ...    
3    2003      11   1296      56   1457      50    N      0    18    38 ...    
4    2003      11   1400      77   1208      71    N      0    30    61 ...    

   Lfga3  Lftm  Lfta  Lor  Ldr  Last  Lto  Lstl  Lblk  Lpf  
0     10    16    22   10   22     8   18     9     2   20  
1     24     9    20   20   25     7   12     8     6   16  
2     26    14    23   31   22     9   12     2     5   23  
3     22     8    15   17   20     9   19     4     3   23  
4     16    17    27   21   15    12   10     7     1   14  

[5 rows x 34 columns]

## Break down tournaments by year

Jayant's Code help

In [57]:
#turn all datasets into DataFrames
teams_df = pd.DataFrame(teams)
seasons_df = pd.DataFrame(seasons)
tourneyCompact_df = pd.DataFrame(tourneyCompactResults)
tourneyDetailed_df = pd.DataFrame(tourneyDetailedResults)
tourneySeeds_df = pd.DataFrame(tourneySeeds)
tourneySlots_df = pd.DataFrame(tourneySlots)
regSeasonCompact_df = pd.DataFrame(regularSeasonCompactResults)
regSeasonDetailed_df = pd.DataFrame(regularSeasonDetailedResults)

In [58]:
# Transform the data, get the seed number for winning and losing team and remove unnecessary columns
tourney_merged_df = pd.merge(left=tourneyCompact_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Wteam'], right_on=['Season', 'Team'])
tourney_merged_df.rename(columns={'Seed' : 'Wseed', 'Team' : 'W_team'}, inplace=True)
tourney_merged_df = pd.merge(left=tourney_merged_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Lteam'], right_on=['Season', 'Team'])
tourney_merged_df.rename(columns={'Seed' : 'Lseed', 'Team' : 'L_team'}, inplace=True)

In [59]:
tourney_sanitized_df = tourney_merged_df[['Season', 'Wteam', 'Lteam', 'Wseed', 'Lseed']].copy()
tourney_sanitized_df['Wseed'] = tourney_sanitized_df['Wseed'].map(lambda x: int(x[1:3]))
tourney_sanitized_df['Lseed'] = tourney_sanitized_df['Lseed'].map(lambda x: int(x[1:3]))
tourney_sanitized_df['Seed_diff'] = tourney_sanitized_df['Wseed'] - tourney_sanitized_df['Lseed']


In [60]:
tourney_winning_df = pd.DataFrame()
tourney_winning_df[['Team1', 'Team2', 'Seed_diff']] = tourney_sanitized_df[['Wteam', 'Lteam', 'Seed_diff']].copy()
tourney_winning_df['result'] = 1
tourney_losing_df = pd.DataFrame()
tourney_losing_df[['Team1', 'Team2', 'Seed_diff']] = tourney_sanitized_df[['Lteam', 'Wteam', 'Seed_diff']].copy()
tourney_losing_df['Seed_diff'] = -tourney_losing_df['Seed_diff']
tourney_losing_df['result'] = 0
tourney_train_final_df = pd.concat((tourney_winning_df, tourney_losing_df))
team_dict = {t: i for i, t in enumerate(tourney_train_final_df.Team1.unique())}

In [61]:
X_train = tourney_train_final_df[['Team1', 'Team2', 'Seed_diff']].copy()
X_train['Team1'] = X_train['Team1'].apply(lambda x: team_dict[x])
X_train['Team2'] = X_train['Team2'].apply(lambda x: team_dict[x])
X_train['Seed_diff'] = X_train['Seed_diff'].values.reshape(-1,1)
Y_train = tourney_train_final_df.result.values

In [62]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_train, Y_train, test_size=0.2, random_state=5)

In [63]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.grid_search import GridSearchCV

n_estimators_list = [5]

etc = ExtraTreesClassifier(random_state=47)
grid = GridSearchCV(estimator=etc, param_grid=dict(n_estimators=n_estimators_list))
grid.fit(X_train, y_train)

print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.n_estimators)

GridSearchCV(cv=None, error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=47, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'n_estimators': [5]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.655792682927
5


Run the below if the above finishes

In [64]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.grid_search import GridSearchCV

n_estimators_list = [5,10,50,100,200,300,400,1000]

etc = ExtraTreesClassifier(random_state=47)
grid = GridSearchCV(estimator=etc, param_grid=dict(n_estimators=n_estimators_list))
grid.fit(X_train, y_train)

print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.n_estimators)

GridSearchCV(cv=None, error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=47, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 50, 100, 200, 300, 400, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.669207317073
400


In [65]:
# Score of best_estimator on the left out data
print("best score is {0}".format(grid.best_score_))

# Print the optimized parameters used in the model selected from grid search
print "Params: ", grid.best_params_

### Assign the best estimator to final Extra Tree classifier
et_clf = grid.best_estimator_

best score is 0.669207317073
Params:  {'n_estimators': 400}


In [76]:
test_et_df = pd.DataFrame()
test_et_df['Season'] = submission_df['id'].apply(lambda x: int(x.split("_")[0]))
test_et_df['Team1'] = submission_df['id'].apply(lambda x: int(x.split("_")[1]))
test_et_df['Team2'] = submission_df['id'].apply(lambda x: int(x.split("_")[2]))

In [77]:
test_et_merged_df = pd.merge(left=test_et_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Team1'], right_on=['Season', 'Team'])
test_et_merged_df.rename(columns={'Seed' : 'Seed1', 'Team' : 'team_1'}, inplace=True)
test_et_merged_df = pd.merge(left=test_et_merged_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Team2'], right_on=['Season', 'Team'])
test_et_merged_df.rename(columns={'Seed' : 'Seed2', 'Team' : 'team_2'}, inplace=True)
test_et_sanitized_df = test_et_merged_df[['Season', 'Team1', 'Team2', 'Seed1', 'Seed2']].copy()
test_et_sanitized_df['Seed1'] = test_et_sanitized_df['Seed1'].map(lambda x: int(x[1:3]))
test_et_sanitized_df['Seed2'] = test_et_sanitized_df['Seed2'].map(lambda x: int(x[1:3]))
test_et_sanitized_df['Seed_diff'] = test_et_sanitized_df['Seed1'] - test_et_sanitized_df['Seed2']

In [82]:
X_et_test = test_et_sanitized_df[['Team1', 'Team2', 'Seed_diff']].copy()
X_et_test['Team1'] = X_et_test['Team1'].apply(lambda x: team_dict[x])
X_et_test['Team2'] = X_et_test['Team2'].apply(lambda x: team_dict[x])
X_et_test['Seed_diff'] = X_et_test['Seed_diff'].values.reshape(-1,1)

In [69]:
test_et_pred_prob = et_clf.predict_proba(X_et_test)[:,1]

In [72]:
final_et_output = pd.DataFrame()
final_et_output['id'] = submission_df['id']
final_et_output['pred'] = test_et_pred_prob
final_et_output.to_csv('./output/et.csv', index=False)